## Preprocessing 

In [ ]:
import nltk
print(nltk.data.find('tokenizers/punkt'))# this address is used in the prepprocess_text_random_forest.py file in the nltk_data_path variable


C:\Users\aj281\AppData\Roaming\nltk_data\tokenizers\punkt


In [2]:
# Step 1: Import the function from your .py file
from preprocess_text_random_forest import run_preprocessing

# Step 2: Call the function — it will return the processed DataFrame
df = run_preprocessing()

# Step 3: Now you can use df for your Random Forest or any further steps
df.head()


100%|██████████| 73/73 [00:26<00:00,  2.80it/s]


,date,text,is_retweet,lang,sentiment,preprocess_text
0,2021-04-12 18:56:10,The Why Behind Microsoft’s $19 Billion Nuance ...,False,en,neutral,The Why Behind Microsoft s Billion Nuance Buy ...
1,2021-04-12 18:55:48,Make a change and an impart on yourself and ot...,False,en,neutral,Make a change and an impart on yourself and ot...
2,2021-04-12 18:55:27,What are the biggest shitcoins? crypto btc bnb...,False,en,neutral,What are the biggest shitcoins crypto btc bnb ...
3,2021-04-12 18:55:22,crypto bitcoin cryptocurrency blockchain btc e...,False,en,neutral,crypto bitcoin cryptocurrency blockchain btc e...
4,2021-04-12 18:55:01,[1D] Bitcoin market is weakly trending up curr...,False,en,negative,Bitcoin market is weakly trending up current m...


## Vectorization of the text

In [3]:
X=df['preprocess_text']
y=df['sentiment']

n=len(X)

split_index=int(n*0.8)
# Split the data into training and testing sets
X_train= X[:split_index]
X_test= X[split_index:]
y_train= y[:split_index]
y_test= y[split_index:]

#sklearn train_test_split is not used because it splits randomly and we want to keep the order of the data for the time series modeling

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(
    stop_words='english',
    lowercase=True, 
    max_features=10000,
    min_df=0.01,
    max_df=0.9,
)



X_train_tfidf=vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)
print("Shape of the feature matrix X: ",X_train_tfidf.shape)
print("Shape of the target vector Y: ",y_train.shape)


Shape of the feature matrix X:  (576213, 167)
Shape of the target vector Y:  (576213,)


## Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy.stats import randint

# Define the parameter distribution
param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(50, 1000),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

# Initialize the classifier
rf = RandomForestClassifier(random_state=42)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,  # Try 20 random combinations
    cv=5,
    scoring='accuracy',  # Change to f1_macro, roc_auc, etc. if needed
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Fit the model
random_search.fit(X_train_tfidf, y_train)

# Get best model
best_rf = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Predict and evaluate
y_pred = best_rf.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 20 candidates, totalling 100 fits
